In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row
from pyspark.sql.functions import *

In [2]:
spark = SparkContext(master='local', appName='DataFrames')
sqlContext = SQLContext(spark)

22/10/01 16:54:27 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface eno2)
22/10/01 16:54:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/01 16:54:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
path_file = '/home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/'

In [4]:
def eliminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [5]:
deportistaError = spark.textFile(path_file + 'deportistaError.csv') \
    .map(lambda l: l.split(','))

deportistaError = deportistaError.mapPartitionsWithIndex(eliminar_encabezado)

In [8]:
deportistaError.take(2)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

In [10]:
deportistaError = deportistaError.map(lambda l : (l[0], l[1], l[2], l[3], l[4], l[5], l[6]))

schema = StructType([
    StructField('deportista_id', StringType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', StringType(), False),
    StructField('edad', StringType(), False),
    StructField('altura', StringType(), False),
    StructField('peso', StringType(), False),
    StructField('equipo_id', StringType(), False)
])

deportistaErrorDF = sqlContext.createDataFrame(deportistaError, schema)
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|      |    |      273|
|            4|Edgar Lindenau Aabye|     1|  34|      |    |      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|    |      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|      |    |      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

In [11]:
from pyspark.sql.functions import udf

In [13]:
def conversionEnteros(valor):
    return int(valor) if len(valor) > 0 else None

conversionEnteros_udf = udf(lambda z: conversionEnteros(z), IntegerType())
sqlContext.udf.register('conversionEnteros_udf', conversionEnteros_udf)

<function __main__.<lambda>(z)>

In [15]:
deportistaErrorDF.select(conversionEnteros_udf('altura').alias('AlturaUDF')).show()

+---------+
|AlturaUDF|
+---------+
|      180|
|      170|
|     null|
|     null|
|      185|
|      188|
|      183|
|      168|
|      186|
|     null|
|      182|
|      172|
|      159|
|      171|
|     null|
|      184|
|      175|
|      189|
|     null|
|      176|
+---------+
only showing top 20 rows



In [16]:
spark.stop()